## Dependencies

In [2]:
# Data Management
import pickle
#import datetime as dt

In [3]:
# Data manipulation dependencies
#import math
import statistics
#import scipy.stats
#import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Custom data handling methods
import prep_IO

In [5]:
# Directory of pre-determined lustre failed jobs
src = './lustre_set.pkl'

In [6]:
search_raw = pickle.load( open( src, "rb" ) )
search_set = [ item for item in search_raw if '2020-09' not in item[1] and '2020-10' not in item[1] ]

#### Generate sample data

In [7]:
sample_n = 2
sample_info = prep_IO.search_sample_n( search_set, sample_n )

In [8]:
# setup friendly keys and fill in search results
keys = list(sample_info["Out"].keys())
sample_hosts = prep_IO.fill_host_info( sample_info )
sample_accts = prep_IO.fill_acct_info( sample_info, keys )

In [9]:
sample_accts

{('comet-06-56',
  '2020-05-27T01:47:29',
  '2020-05-27T05:30:13',
  '33659176'): {'JobID': '33659176', 'User': 'djone128', 'Account': 'iwa117', 'Start': '2020-05-27T01:47:29', 'End': '2020-05-27T05:30:13', 'Submit': '2020-05-26T18:22:46', 'Partition': 'compute', 'Timelimit': '2-00:00:00', 'JobName': 'MNCO', 'State': 'FAILED', 'NNodes': '8', 'ReqCPUS': '64', 'NodeList\n': 'comet-06-56,comet-07-14,comet-11-24,comet-14-71,comet-15-34,comet-20-[09,53],comet-22-30\n', 'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-06-56.sdsc.edu/1590543637.gz'], 'Single JobID': False},
 ('comet-18-71',
  '2020-05-12T06:58:43',
  '2020-05-13T17:08:24',
  '33283105'): {'JobID': '33283105', 'User': 'nyporko', 'Account': 'jsu101', 'Start': '2020-05-12T06:58:43', 'End': '2020-05-13T17:08:24', 'Submit': '2020-05-11T10:12:31', 'Partition': 'compute', 'Timelimit': '1-23:59:00', 'JobName': '7umbrella7', 'State': 'FAILED', 'NNodes': '4', 'ReqCPUS': '96', 'NodeList\n': 'comet-05-23,comet-18-71

## Data

1. Size
2. Missingness
3. Shape
4. Central Tendency
5. Variability

- first level keys: search labels
- second level keys: ['Acct Info', 'Host Info', 'Source Files']
- 'Host Info' keys: ['Timely Data', 'Schemas', 'Specs']


In [ ]:
# Token search from file lines
#lines = prep_IO.unzip_txt( check["Source"][0] )
#token = 'ps'
#
#for line in lines:
#    if token in line:
#        print(line)

### Preview Sample

In [11]:
sample_info["Sample"]

[('comet-06-56', '2020-05-27T01:47:29', '2020-05-27T05:30:13', '33659176'),
 ('comet-18-71', '2020-05-12T06:58:43', '2020-05-13T17:08:24', '33283105')]

In [14]:
sample_tar = keys[0]
sample_accts[ sample_tar ]

{'JobID': '33659176',
 'User': 'djone128',
 'Account': 'iwa117',
 'Start': '2020-05-27T01:47:29',
 'End': '2020-05-27T05:30:13',
 'Submit': '2020-05-26T18:22:46',
 'Partition': 'compute',
 'Timelimit': '2-00:00:00',
 'JobName': 'MNCO',
 'State': 'FAILED',
 'NNodes': '8',
 'ReqCPUS': '64',
 'NodeList\n': 'comet-06-56,comet-07-14,comet-11-24,comet-14-71,comet-15-34,comet-20-[09,53],comet-22-30\n',
 'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-06-56.sdsc.edu/1590543637.gz'],
 'Single JobID': False}

In [17]:
sample_hosts[ sample_tar ][ "Specs" ]

['tacc_stats 2.3.4',
 'hostname comet-06-56.sdsc.edu',
 'uname Linux x86_64 3.10.0-957.12.2.el7.x86_64 #1 SMP Tue May 14 21:24:32 UTC 2019',
 'uptime 6675063']

In [23]:
sample_hosts[ sample_tar ][ "Source" ]

['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-06-56.sdsc.edu/1590543637.gz']

In [24]:
sample_hosts[ sample_tar ][ "Single JobID" ]

False

## Plot stuff

plot templates

In [ ]:
## Histogram
##hist, bin_edges = np.histogram(x, bins=10)
#fig, ax = plt.subplots()
#ax.hist(x, bin_edges, cumulative=False)
#ax.set_xlabel('x')
#ax.set_ylabel('Frequency')
#plt.show()
#
## Pie Chart
## x, y, z = 128, 256, 1024
#fig, ax = plt.subplots()
#ax.pie((x, y, z), labels=('x', 'y', 'z'), autopct='%1.1f%%')
#plt.show()
#
## Horizontal Bar Chart
##x = np.arange(21)
##y = np.random.randint(21, size=21)
##err = np.random.randn(21)
#fig, ax = plt.subplots())
#ax.bar(x, y, yerr=err)
#ax.set_xlabel('x')
#ax.set_ylabel('y')
#plt.show()
#
## Scatterplot w/ Line
##x = np.arange(21)
##y = 5 + 2 * x + 2 * np.random.randn(21)
##slope, intercept, r, *__ = scipy.stats.linregress(x, y)
##line = f'Regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
#fig, ax = plt.subplots()
#ax.plot(x, y, linewidth=0, marker='s', label='Data points')
#ax.plot(x, intercept + slope * x, label=line)
#ax.set_xlabel('x')
#ax.set_ylabel('y')
#ax.legend(facecolor='white')
#plt.show()
#
## Seaborn heatmap w/ 2 var
##Index= ['aaa', 'bbb', 'ccc', 'ddd', 'eee']
##Cols = ['A', 'B', 'C', 'D']
##data = abs(np.random.randn(5, 4))
##df = DataFrame(data, index=Index, columns=Cols)
#sns.heatmap(df, annot=True)

In [ ]:
# SAVE template
#save_thing = test_dict
#f = open("save_thing_name.pkl","wb")
#pickle.dump( save_thing,f )
#f.close()

# read in search_out from saved dict in src_file
#save_thing = pickle.load( open( "save_thing_name.pkl", "rb" ) )
#keys = list(save_thing.keys())
#len(keys)

# Analysis

#### Setup Structures

GOAL:
Format data for a visualization with X-axis as time, y-axis as cat value, and each cat as a plotted line

In [28]:
# All valid 'Timely Data' occurences in dataset
data_lists = [ sample_hosts[key]['Data'] for key in keys ]

# friendlllyy data
df_a = pd.DataFrame( data_lists[0] )
df_b = pd.DataFrame( data_lists[1] )

# timelines for each
times_a = list( df_a[4].unique() )
times_b = list( df_b[4].unique() )

#### Identify Variable behavior

In [29]:
target = data_lists[0]

In [30]:
timedata_by_label = prep_IO.labeled_data( target )
timedata_keys = list( timedata_by_label.keys() )

In [31]:
cat_counts = { key:len(value) for key,value in timedata_by_label.items() }
base = statistics.mode( cat_counts.values() )

std_cats = []
irreg_cats = []

for cat,count in cat_counts.items():
    if count != base:
        if cat not in irreg_cats:
            irreg_cats.append( cat )
    else:
        if cat not in std_cats:
            std_cats.append( cat )
            
    #print( f"{cat} recorded {count} times." )

### General Descriptive Statistics

Data manipulation:
* If host recorded value at `t_0` and increased with each `t_i`, record base value and decrease each `t_i` by base value before performing descriptive operations
    * Certain categories record values that do not necessarily reset between jobs.
* If all `t_i` represent same value, move category to "Static" and do not include in descriptive operations
* `std_cats` appear for each `t_i`
* `irreg_cats` appear sporadically throughout run

In [44]:
descriptives = prep_IO.get_host_descriptives( timedata_by_label, timedata_keys )
std_desc = { key:value for key,value in descriptives.items() if key in std_cats }
irr_desc = { key:value for key,value in descriptives.items() if key in irreg_cats }
static_cats = descriptives["Static"]
n_static = len(static_cats)

#prep_IO.quick_save( descriptives, label='lustre_set_descriptives' )

In [56]:
print( f"Static categories:\t { n_static }")
print( f"Unique categories:\t {len(descriptives) - 1}" )
print( f"\tStandard:\t { len(std_desc) }")
print( f"\tIrregular:\t { len(irr_desc) }")

Static categories:	 978
Unique categories:	 625
	Standard:	 573
	Irregular:	 52


# Manual cat inspection

In [46]:
std_desc

{('block', 'md1', 'rd_ios'): {'Count': 24,
  'Min': 0.0,
  'Max': 11481.0,
  'Mode': 5985.0,
  'Quartiles': array([2360., 5982., 5985.]),
  'Mean': 4633.416666666667,
  'Std. Dev': 2386.5617862779545,
  'Skew': 0.46157043860890556,
  'Nonzero Start': False,
  'Starting Value': 1488857.0},
 ('block', 'md1', 'rd_sectors(U=512B)'): {'Count': 24,
  'Min': 0.0,
  'Max': 285696.0,
  'Mode': 233664.0,
  'Quartiles': array([134432., 233640., 233664.]),
  'Mean': 189183.0,
  'Std. Dev': 64051.920868932575,
  'Skew': -1.0124381786275636,
  'Nonzero Start': False,
  'Starting Value': 80061730.0},
 ('block', 'md1', 'wr_ios'): {'Count': 24,
  'Min': 0.0,
  'Max': 15527.0,
  'Quartiles': array([ 4234.5 ,  9048.5 , 12557.25]),
  'Mean': 8441.916666666666,
  'Std. Dev': 4791.306606037183,
  'Skew': -0.12302495084044883,
  'Nonzero Start': False,
  'Starting Value': 20472928.0},
 ('block', 'md1', 'wr_sectors(U=512B)'): {'Count': 24,
  'Min': 0.0,
  'Max': 153000.0,
  'Quartiles': array([ 37396.,  94492

In [47]:
irr_desc

{('proc', 'cvmfs2/11394/0-23/0-1', 'VmRSS(U=kB)'): {'Count': 3,
  'Min': 0.0,
  'Max': 12.0,
  'Mode': 12.0,
  'Quartiles': array([ 6., 12., 12.]),
  'Mean': 8.0,
  'Std. Dev': 5.656854249492381,
  'Skew': -0.7071067811865475,
  'Nonzero Start': False,
  'Starting Value': 39512.0},
 ('proc', 'cvmfs2/10999/0-23/0-1', 'VmHWM(U=kB)'): {'Count': 5,
  'Min': 0.0,
  'Max': 336.0,
  'Mode': 0.0,
  'Quartiles': array([0., 0., 0.]),
  'Mean': 67.2,
  'Std. Dev': 134.4,
  'Skew': 1.5,
  'Nonzero Start': False,
  'Starting Value': 39700.0},
 ('proc', 'cvmfs2/10999/0-23/0-1', 'VmRSS(U=kB)'): {'Count': 5,
  'Min': 0.0,
  'Max': 2324.0,
  'Mode': 12.0,
  'Quartiles': array([12., 12., 12.]),
  'Mean': 472.0,
  'Std. Dev': 926.0116629935068,
  'Skew': 1.499905376713693,
  'Nonzero Start': False,
  'Starting Value': 37712.0},
 ('proc', 'cvmfs2/10999/0-23/0-1', 'VmPTE(U=kB)'): {'Count': 5,
  'Min': 0.0,
  'Max': 4.0,
  'Mode': 0.0,
  'Quartiles': array([0., 0., 0.]),
  'Mean': 0.8,
  'Std. Dev': 1.6,
  

## Graphs

In [57]:
times_a

['1590569253.234787',
 '1590569853.371716',
 '1590570453.556353',
 '1590571053.710548',
 '1590571653.893002',
 '1590572254.067723',
 '1590572854.241089',
 '1590573454.414888',
 '1590574054.540724',
 '1590574654.712068',
 '1590575254.884523',
 '1590575855.057720',
 '1590576455.232134',
 '1590577055.377721',
 '1590577655.550380',
 '1590578255.724051',
 '1590578855.897470',
 '1590579456.045717',
 '1590580056.210531',
 '1590580656.371723',
 '1590581256.545053',
 '1590581856.710543',
 '1590582456.883440',
 '1590582617.617811']

In [60]:
data_lists[0]

[('block', 'md0', 'rd_ios', '6065', '1590569253.234787', '33659176'),
 ('block', 'md0', 'rd_merges', '0', '1590569253.234787', '33659176'),
 ('block',
  'md0',
  'rd_sectors(U=512B)',
  '308748',
  '1590569253.234787',
  '33659176'),
 ('block', 'md0', 'rd_ticks(U=ms)', '0', '1590569253.234787', '33659176'),
 ('block', 'md0', 'wr_ios', '2358', '1590569253.234787', '33659176'),
 ('block', 'md0', 'wr_merges', '0', '1590569253.234787', '33659176'),
 ('block', 'md0', 'wr_sectors(U=512B)', '92', '1590569253.234787', '33659176'),
 ('block', 'md0', 'wr_ticks(U=ms)', '0', '1590569253.234787', '33659176'),
 ('block', 'md0', 'in_flight', '0', '1590569253.234787', '33659176'),
 ('block', 'md0', 'io_ticks(U=ms)', '0', '1590569253.234787', '33659176'),
 ('block', 'md1', 'rd_ios', '1488857', '1590569253.234787', '33659176'),
 ('block', 'md1', 'rd_merges', '0', '1590569253.234787', '33659176'),
 ('block',
  'md1',
  'rd_sectors(U=512B)',
  '80061730',
  '1590569253.234787',
  '33659176'),
 ('block', '

In [61]:
data_lists[1]

[('block', 'md0', 'rd_ios', '9141', '1589291926.478480', '33283105'),
 ('block', 'md0', 'rd_merges', '0', '1589291926.478480', '33283105'),
 ('block',
  'md0',
  'rd_sectors(U=512B)',
  '465950',
  '1589291926.478480',
  '33283105'),
 ('block', 'md0', 'rd_ticks(U=ms)', '0', '1589291926.478480', '33283105'),
 ('block', 'md0', 'wr_ios', '249', '1589291926.478480', '33283105'),
 ('block', 'md0', 'wr_merges', '0', '1589291926.478480', '33283105'),
 ('block', 'md0', 'wr_sectors(U=512B)', '76', '1589291926.478480', '33283105'),
 ('block', 'md0', 'wr_ticks(U=ms)', '0', '1589291926.478480', '33283105'),
 ('block', 'md0', 'in_flight', '0', '1589291926.478480', '33283105'),
 ('block', 'md0', 'io_ticks(U=ms)', '0', '1589291926.478480', '33283105'),
 ('block', 'md1', 'rd_ios', '1833024', '1589291926.478480', '33283105'),
 ('block', 'md1', 'rd_merges', '0', '1589291926.478480', '33283105'),
 ('block',
  'md1',
  'rd_sectors(U=512B)',
  '72781890',
  '1589291926.478480',
  '33283105'),
 ('block', 'm